In [ ]:
#Mount google drive so Colab notebook can access the files stored on it. We may not need to run this repeatedly
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

random_state = 42

In [ ]:
path = '/content/drive/Shareddrives/SIADS699-Fall2022-Learning-Opportunity/'

In [ ]:
# path = ''

In [ ]:
coi = pd.read_csv(path+'data_raw/COI_raw.csv', dtype={'geoid':'str'})

In [ ]:
pop = pd.read_csv(path+'data_raw/DECENNIALPL2010.P1-Data.csv', skiprows=[1], dtype={'P001001': 'str', 'P001001ERR': 'str'})

In [ ]:
cross_ref = pd.read_excel(path+'data_raw/grf15_lea_tract.xlsx', dtype={'LEAID': 'str', 'TRACT': 'str'})

In [ ]:
# Split COI data by year (2010 or 2015)
def coi_split_year(coi):

    coi_2010 = coi[coi['year'] == 2010]
    coi_2015 = coi[coi['year'] == 2015]

    return coi_2010, coi_2015

In [ ]:
# Split data from the cross reference between LEAID and census TRACT to get keys for splitting
# both COI and SEDA data
def leaid_split(cross_ref, year):
    leaid = cross_ref['LEAID'].unique()

    # Split to get test and train
    # Use year as random state to get different splits for each year
    leaid_train, leaid_test = train_test_split(leaid, test_size=0.2, random_state=year)

    # Write LEAIDs to csv for use with SEDA data
    leaid_train_ser = pd.Series(leaid_train)
    filename = path + 'data_inprocess/leaids_train_' + str(year) + '.csv'
    leaid_train_ser.to_csv(filename)

    leaid_test_ser = pd.Series(leaid_test)
    filename = path + 'data_inprocess/leaids_test_' + str(year) + '.csv'
    leaid_test_ser.to_csv(filename)

    # Take training and test sets of cross_ref dataset by matching with LEAIDs
    cross_ref_train = cross_ref[cross_ref['LEAID'].isin(leaid_train)]
    cross_ref_test = cross_ref[cross_ref['LEAID'].isin(leaid_test)]

    return cross_ref_train, cross_ref_test

In [ ]:
# Preprocess cross_ref to calculate percentage of tract land area per school district
def preprocess_cross_ref(cross_ref):

    # Groupby tract to get total tract land area
    cross_ref['LANDAREA_TOT'] = cross_ref.groupby('TRACT')['LANDAREA'].transform('sum')

    # Divide tract land area per district by total tract land area to get percent
    cross_ref['LANDAREA_PERC'] = cross_ref['LANDAREA'] / cross_ref['LANDAREA_TOT']

    return cross_ref

In [ ]:
# Process and join COI data with census population data to get total population 
# to use in weighting COI indicators
def preprocess_coi(coi, pop):

    # Strip '1400000US' from beginning of GEO_ID field
    pop.loc[:, 'TRACT'] = pop['GEO_ID'].str.replace('1400000US', '')

    # Remove the revision indicator string from the total population value
    pop.loc[:, 'pop_total'] = pop['P001001'].str.replace(r'\(r[0-9]+\)', '', regex=True)

    # Just take new geoid column with the population total
    pop_tract = pop[['TRACT', 'pop_total']].copy()

    # Set population total value data type to int
    pop_tract.loc[:, 'pop_total'] = pop_tract['pop_total'].astype('int64')

    # Rename COI columns to pop_child (as opposed to pop_total from the census) and geoid to TRACT for joining
    coi = coi.rename(columns={'pop': 'pop_child', 'geoid': 'TRACT'})

    # Merge COI and census population
    coi = coi.merge(pop_tract, on='TRACT')

    return coi

In [ ]:
# Train-test split COI data
def train_test_split_coi(cross_ref, coi, pop, year):

    # Preprocess the cross_ref data to get percent land area of each tract in a
    # given school district
    cross_ref_proc = preprocess_cross_ref(cross_ref)

    # Subsection the cross_ref data based on spliting by LEAID
    cross_ref_train, cross_ref_test = leaid_split(cross_ref_proc, year)

    # Preprocess the COI data
    coi_proc = preprocess_coi(coi, pop)

    # Subset COI data by inner-joining with cross_ref subsections
    coi_dist_train = cross_ref_train.merge(coi_proc, on='TRACT')
    coi_dist_test = cross_ref_test.merge(coi_proc, on='TRACT')

    return coi_dist_train, coi_dist_test

In [ ]:
# Process without splitting for various clustering methods
def process_coi_all(cross_ref, coi, pop):

    # Preprocess the cross_ref data to get percent land area of each tract in a
    # given school district
    cross_ref_proc = preprocess_cross_ref(cross_ref)

    # Preprocess the COI data
    coi_proc = preprocess_coi(coi, pop)

    # Merge COI data with cross_ref
    coi_dist_all = cross_ref_proc.merge(coi_proc, on='TRACT')

    return coi_dist_all

In [ ]:
# Create population-weighted averages for COI indicators, scaled by the percent of tract land area
# in the school district
def weight_coi(coi_indicators, ind_column_names):

    # Non-indicator column names to be merged back in after weighting
    non_ind_cols = ['LEAID', 'NAME_LEA15', 'TRACT', 'year', 'pop_child', 'pop_total', 'pop_scaled']

    # Scale the total population by the percentage of tract land area that is in a given 
    # school district
    coi_indicators['pop_scaled'] = coi_indicators['pop_total'] * coi_indicators['LANDAREA_PERC']

    # Weight the indicators by the scaled population from the census data
    coi_weighted = coi_indicators[ind_column_names].multiply(coi_indicators['pop_scaled'], axis='index')

    # Merge back in the LEA, tract, year, and population columns
    coi_weighted = coi_indicators.loc[:, non_ind_cols].merge(coi_weighted, left_index=True, right_index=True)

    return coi_weighted

In [ ]:
# Group weighted indicators into school districts, then divide by total population to get a 
# weighted average
def group_coi(coi_district, ind_column_names):

    # Group by school district ID/name and year
    coi_grouped = coi_district.groupby(['LEAID', 'NAME_LEA15', 'year']).sum()

    # Divide the weighted indicators by the total population of the district to get the weighted average
    coi_grouped.loc[:, ind_column_names] = coi_grouped.loc[:, ind_column_names].divide(coi_grouped['pop_scaled'], axis='index')

    # Reset the multi-index
    coi_grouped = coi_grouped.reset_index()

    return coi_grouped

In [25]:
def process_coi(coi, pop, cross_ref, year, write=False):

    # Preprocess cross-ref, split cross-ref LEAIDs, preprocess COI, split COI/cross-ref into train/validation/test
    coi_dist_train, coi_dist_test = train_test_split_coi(cross_ref, coi, pop, year)

    # Identify the numeric indicator columns
    ind_column_names = ['ED_APENR', 'ED_ATTAIN', 'ED_COLLEGE', 'ED_ECENROL', 'ED_HSGRAD', 'ED_MATH', 
                        'ED_READING','ED_SCHPOV', 'ED_TEACHXP', 'ED_PRXECE', 'ED_PRXHQECE', 
                        'HE_FOOD', 'HE_GREEN', 'HE_HEAT', 'HE_HLTHINS', 'HE_OZONE', 'HE_PM25',
                        'HE_VACANCY', 'HE_WALK', 'HE_SUPRFND', 'HE_RSEI', 'SE_POVRATE', 'SE_PUBLIC', 
                        'SE_HOME', 'SE_OCC', 'SE_MHE', 'SE_EMPRAT', 'SE_JOBPROX', 'SE_SINGLE']

    # Non-indicator columns
    non_ind_column_names = ['LEAID', 'NAME_LEA15', 'TRACT', 'COUNT', 'LANDAREA_PERC', 'year', 'pop_child', 'pop_total']

    # Process training data

    # Get the numeric indicator columns
    X_train = coi_dist_train[ind_column_names]

    # Make a pipeline for processing
    # Use median for imputer strategy because some of the variable distributions are highly skewed
    pipe = Pipeline([('impute', SimpleImputer(strategy='median')), ('scale', StandardScaler())])

    # Fit/transform just the numeric indicator columns
    X_transformed = pipe.fit_transform(X_train)

    # Reconstitute the dataframe with transformed data
    X_trans_df = pd.DataFrame(X_transformed, columns=ind_column_names)

    # Get non-indicator columns from training df
    coi_cols = coi_dist_train[non_ind_column_names]

    # Merge non-indicator and transformed columns
    coi_dist_train_prep = coi_cols.merge(X_trans_df, left_index=True, right_index=True)

    # Weight indicators by total population and land area
    coi_dist_train_prep = weight_coi(coi_dist_train_prep, ind_column_names)

    # Group COI data by school district
    coi_grp_dist_train = group_coi(coi_dist_train_prep, ind_column_names)


    # Write the data out to csv
    if write:
        filename = path + 'data_inprocess/coi_weighted_train_' + str(year) + '.csv'
        coi_dist_train_prep.to_csv(filename)

        filename = path + 'data_cleaned/coi_district_grouped_train_' + str(year) + '.csv'
        coi_grp_dist_train.to_csv(filename)

    # Process test data

    # Get the numeric indicator columns
    X_test = coi_dist_test[ind_column_names]

    # Just transform just the numeric indicator columns (use pipeline fitted above)
    X_transformed = pipe.transform(X_test)

    # Reconstitute the dataframe with transformed data
    X_trans_df = pd.DataFrame(X_transformed, columns=ind_column_names)

    # Get non-indicator columns from training df
    coi_cols = coi_dist_test[non_ind_column_names]

    # Merge non-indicator and transformed columns
    coi_dist_test_prep = coi_cols.merge(X_trans_df, left_index=True, right_index=True)

    # Weight indicators by total population and land area
    coi_dist_test_prep = weight_coi(coi_dist_test_prep, ind_column_names)

    # Group COI data by school district
    coi_grp_dist_test = group_coi(coi_dist_test_prep, ind_column_names)


    # Write the test data out to csv
    if write:
        filename = path + 'data_inprocess/coi_weighted_test_' + str(year) + '.csv'
        coi_dist_test_prep.to_csv(filename)

        filename = path + 'data_cleaned/coi_district_grouped_test_' + str(year) + '.csv'
        coi_grp_dist_test.to_csv(filename)

    return coi_grp_dist_train, coi_grp_dist_test

In [23]:
# Split COI by year (2010, 2015)
coi_2010, ooi_2015 = coi_split_year(coi)

In [26]:
coi_grp_dist_train_2010, coi_grp_dist_test_2010 = process_coi(coi_2010, pop, cross_ref, year=2010, write=True)
coi_grp_dist_train_2015, coi_grp_dist_test_2015 = process_coi(ooi_2015, pop, cross_ref, year=2015, write=True)

# Older layout

In [ ]:
# Preprocess cross-ref, split cross-ref LEAIDs, preprocess COI, split COI/cross-ref into train/validation/test
coi_dist_train, coi_dist_test = train_test_split_coi(cross_ref, coi, pop)

In [ ]:
# Identify the numeric indicator columns
ind_column_names = ['ED_APENR', 'ED_ATTAIN', 'ED_COLLEGE', 'ED_ECENROL', 'ED_HSGRAD', 'ED_MATH', 
                    'ED_READING','ED_SCHPOV', 'ED_TEACHXP', 'ED_PRXECE', 'ED_PRXHQECE', 
                    'HE_FOOD', 'HE_GREEN', 'HE_HEAT', 'HE_HLTHINS', 'HE_OZONE', 'HE_PM25',
                    'HE_VACANCY', 'HE_WALK', 'HE_SUPRFND', 'HE_RSEI', 'SE_POVRATE', 'SE_PUBLIC', 
                    'SE_HOME', 'SE_OCC', 'SE_MHE', 'SE_EMPRAT', 'SE_JOBPROX', 'SE_SINGLE']

# Non-indicator columns
non_ind_column_names = ['LEAID', 'NAME_LEA15', 'TRACT', 'COUNT', 'LANDAREA_PERC', 'year', 'pop_child', 'pop_total']

In [ ]:
# Process training data

# Get the numeric indicator columns
X_train = coi_dist_train[ind_column_names]

# Make a pipeline for processing
# Use median for imputer strategy because some of the variable distributions are highly skewed
pipe = Pipeline([('impute', SimpleImputer(strategy='median')), ('scale', StandardScaler())])

# Fit/transform just the numeric indicator columns
X_transformed = pipe.fit_transform(X_train)

# Reconstitute the dataframe with transformed data
X_trans_df = pd.DataFrame(X_transformed, columns=ind_column_names)

# Get non-indicator columns from training df
coi_cols = coi_dist_train[non_ind_column_names]

# Merge non-indicator and transformed columns
coi_dist_train_prep = coi_cols.merge(X_trans_df, left_index=True, right_index=True)

# Weight indicators by total population and land area
coi_dist_train_prep = weight_coi(coi_dist_train_prep, ind_column_names)

# Group COI data by school district
coi_grp_dist_train = group_coi(coi_dist_train_prep, ind_column_names)

In [ ]:
# # Write the data out to csv
# coi_dist_train_prep.to_csv(path+'data_inprocess/coi_weighted_train.csv')
# coi_grp_dist_train.to_csv(path+'data_cleaned/coi_district_grouped_train.csv')

In [ ]:
print(coi_grp_dist_train.shape)
coi_grp_dist_train.sample(5)

(21692, 35)


,LEAID,NAME_LEA15,year,pop_child,pop_total,pop_scaled,ED_APENR,ED_ATTAIN,ED_COLLEGE,ED_ECENROL,...,HE_SUPRFND,HE_RSEI,SE_POVRATE,SE_PUBLIC,SE_HOME,SE_OCC,SE_MHE,SE_EMPRAT,SE_JOBPROX,SE_SINGLE
3882,1704140,Arlington Heights School District 25,2010,16181,68574,25092.632125,1.936663,1.413230,0.285633,0.856029,...,-0.240462,1.138179,-0.998462,-0.869165,0.565324,1.171377,0.939732,0.491668,0.382479,-1.112981
15056,3904885,Maysville Local School District,2010,6417,26744,12166.074661,-0.821982,-1.021244,-0.926039,0.440804,...,-0.240462,0.340378,0.525360,0.841420,0.452963,-1.136734,-0.701327,-0.244411,-0.019685,0.408404
7413,2400180,Caroline County Public Schools,2015,7819,33066,33066.000000,0.481088,-0.613868,-0.742539,-0.798459,...,-0.240462,-0.522103,0.158200,0.546321,0.176696,-0.536586,-0.280837,0.104734,0.952692,0.229158
8957,2703750,Battle Lake Public School District,2015,4148,19855,3309.727906,-0.994588,-0.113533,-1.416820,-0.194186,...,-0.240462,-0.679701,-0.581114,-0.657398,0.873756,-0.075986,-0.167081,0.789226,-0.087976,-0.671423
21185,5506270,Hayward Community School District,2015,4068,21292,11219.957627,0.004819,-0.154986,-2.257819,-0.128722,...,-0.240462,0.041838,0.131163,0.464243,0.186597,-0.181945,-0.563620,0.358867,-0.483709,0.440100


In [ ]:
# Process test data

# Get the numeric indicator columns
X_test = coi_dist_test[ind_column_names]

# Just transform just the numeric indicator columns (use pipeline fitted above)
X_transformed = pipe.transform(X_test)

# Reconstitute the dataframe with transformed data
X_trans_df = pd.DataFrame(X_transformed, columns=ind_column_names)

# Get non-indicator columns from training df
coi_cols = coi_dist_test[non_ind_column_names]

# Merge non-indicator and transformed columns
coi_dist_test_prep = coi_cols.merge(X_trans_df, left_index=True, right_index=True)

# Weight indicators by total population and land area
coi_dist_test_prep = weight_coi(coi_dist_test_prep, ind_column_names)

# Group COI data by school district
coi_grp_dist_test = group_coi(coi_dist_test_prep, ind_column_names)

In [ ]:
# # Write the test data out to csv
# coi_dist_test_prep.to_csv(path+'data_inprocess/coi_weighted_test.csv')
# coi_grp_dist_test.to_csv(path+'data_cleaned/coi_district_grouped_test.csv')

In [ ]:
print(coi_grp_dist_test.shape)
coi_grp_dist_test.sample(5)

(5426, 35)


,LEAID,NAME_LEA15,year,pop_child,pop_total,pop_scaled,ED_APENR,ED_ATTAIN,ED_COLLEGE,ED_ECENROL,...,HE_SUPRFND,HE_RSEI,SE_POVRATE,SE_PUBLIC,SE_HOME,SE_OCC,SE_MHE,SE_EMPRAT,SE_JOBPROX,SE_SINGLE
1982,2512870,Whately School District,2010,836,5180,1517.457024,-0.273099,1.366605,2.676653,-0.004994,...,-0.240462,0.248446,-0.115585,-0.775306,-0.265554,0.668433,0.119512,0.852083,-0.761727,-0.633209
3691,3904553,Mount Gilead Exempted Village School District,2015,7373,29349,7834.279246,-1.050512,-0.720314,-1.148753,0.305441,...,-0.240462,0.128968,-0.659442,0.424039,0.342042,-0.263870,-0.440644,0.381233,-0.353605,0.009012
1900,2503540,Chelsea School District,2010,9506,35177,35177.000000,0.032930,-0.803696,1.225586,-0.374192,...,-0.240462,0.501230,0.855916,1.368029,-1.808178,-1.257166,-0.448048,0.037766,0.690778,1.123625
2128,2627390,Palo Community School District,2010,3060,13769,1365.701071,-0.558738,-1.076798,-0.736116,-0.241298,...,-0.240462,-0.210991,-0.028195,0.311443,0.847158,-0.667458,-0.329513,-0.664820,0.648341,-0.310509
3252,3607530,Churchville-Chili Central School District,2010,10086,45230,21821.725543,0.886235,0.448774,1.419206,0.143797,...,-0.240462,-0.075904,-0.809198,-0.744442,0.629728,0.540853,0.364440,0.800150,-0.834509,-0.443962


In [ ]:
# Process total dataset for DBSCAN and other transductive clustering methods

# Merge COI data with pop and cross-ref
coi_dist_all = process_coi_all(cross_ref, coi, pop)

# Get the numeric indicator columns
X_all = coi_dist_all[ind_column_names]

# Make a pipeline for processing
# Use median for imputer strategy because some of the variable distributions are highly skewed
pipe = Pipeline([('impute', SimpleImputer(strategy='median')), ('scale', StandardScaler())])

# Fit/transform just the numeric indicator columns
X_trans_all = pipe.fit_transform(X_all)

# Reconstitute the dataframe with transformed data
X_trans_all_df = pd.DataFrame(X_trans_all, columns=ind_column_names)

# Get non-indicator columns from training df
coi_cols_all = coi_dist_all[non_ind_column_names]

# Merge non-indicator and transformed columns
coi_dist_all_prep = coi_cols_all.merge(X_trans_all_df, left_index=True, right_index=True)

# Weight indicators by total population and land area
coi_dist_all_prep = weight_coi(coi_dist_all_prep, ind_column_names)

# Group COI data by school district
coi_grp_dist_all = group_coi(coi_dist_all_prep, ind_column_names)

In [ ]:
# # Write it out to csv
# coi_dist_all_prep.to_csv(path+'data_inprocess/coi_weighted_all.csv')
# coi_grp_dist_all.to_csv(path+'data_cleaned/coi_district_grouped_all.csv')

In [ ]:
print(coi_grp_dist_all.shape)
coi_grp_dist_all.sample(5)

(27118, 35)


,LEAID,NAME_LEA15,year,pop_child,pop_total,pop_scaled,ED_APENR,ED_ATTAIN,ED_COLLEGE,ED_ECENROL,...,HE_SUPRFND,HE_RSEI,SE_POVRATE,SE_PUBLIC,SE_HOME,SE_OCC,SE_MHE,SE_EMPRAT,SE_JOBPROX,SE_SINGLE
20730,4203480,Berwick Area School District,2010,5120,23878,22104.038852,-0.561162,-0.763083,0.289325,0.248079,...,-0.241925,0.333381,-0.215539,-0.137203,0.059119,-0.717828,-0.572460,0.088507,-0.509418,0.164687
8287,2100078,Fort Campbell Dependent Schools,2015,5846,21237,13692.206305,-0.136544,-0.597123,-1.715925,-0.231197,...,-0.241925,0.622564,-0.145614,-0.370781,-2.195491,0.793702,-0.603513,-4.678343,-1.161441,-0.977698
1694,0606580,Butte Valley Unified School District,2010,1861,8927,3304.057945,-1.193791,-0.581144,-1.392896,0.902100,...,-0.241925,-0.763860,0.314235,-0.166635,-0.138684,-0.335609,-0.934840,-0.900837,-0.848801,-0.016418
16165,3603930,Ballston Spa Central School District,2015,9750,42978,28873.856505,0.671065,0.672509,0.612164,0.677776,...,-0.241925,0.249726,-0.719223,-0.225051,0.085687,0.633926,0.498126,0.799164,-0.442395,-0.232374
15855,3418180,Woodbury Heights Borough School District,2015,618,3055,1527.500000,-0.630718,-0.305775,0.454801,-0.501021,...,-0.241925,0.703782,-1.028454,-0.273576,0.940988,0.267938,0.652131,-0.047890,0.282577,0.043675
